# 第7回課題
2段階の train, valid, test にわける交差検証の自前の実装
（パッケージにしたり関数にしたりするのは自由です．解答はしていません）

https://scikit-learn.org/stable/modules/cross_validation.html　で様々なパターンについて述べられています

# 注意事項
train, valid, test で取ってくるサンプル（id）が一致しないということを必ず確認してください

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import random
from sklearn.datasets import load_wine
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

In [2]:
data = load_wine()

In [3]:
wine_df = pd.DataFrame(min_max_scaler.fit_transform(data["data"]), columns=data["feature_names"])
wine_df = pd.concat([wine_df, pd.DataFrame(data["target"], columns=["target"])], axis=1)
print(wine_df.shape)
wine_df.head()

(178, 14)


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,0.842105,0.191700,0.572193,0.257732,0.619565,0.627586,0.573840,0.283019,0.593060,0.372014,0.455285,0.970696,0.561341,0
1,0.571053,0.205534,0.417112,0.030928,0.326087,0.575862,0.510549,0.245283,0.274448,0.264505,0.463415,0.780220,0.550642,0
2,0.560526,0.320158,0.700535,0.412371,0.336957,0.627586,0.611814,0.320755,0.757098,0.375427,0.447154,0.695971,0.646933,0
3,0.878947,0.239130,0.609626,0.319588,0.467391,0.989655,0.664557,0.207547,0.558360,0.556314,0.308943,0.798535,0.857347,0
4,0.581579,0.365613,0.807487,0.536082,0.521739,0.627586,0.495781,0.490566,0.444795,0.259386,0.455285,0.608059,0.325963,0


# モデル（今回は決定木を利用）

In [4]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(random_state=0)

# seed の設定

In [28]:
from collections import Counter
seed = 0
random.seed(seed)
np.random.seed(seed)

# k-fold の k は変数で扱うこと

In [29]:
k = 5

# train (valid 含む), test の分割

In [30]:
l=list(range(len(wine_df)))#インデックスのlistを作成
random.shuffle(l)#indexlistのシャッフル
train_size=int(len(l)*0.8)
train=wine_df.iloc[l[:train_size],:]#8割をtrainvalidに使用
test=wine_df.iloc[l[train_size:],:]#2割をtestに使用

# 交差検証
train, valid を分割しつつ valid をテスト

In [31]:
from sklearn.metrics import accuracy_score # 回答はaccuracy だけにしています．ここも特にしていないです

In [32]:
feature_list=wine_df.columns
feature_list=feature_list.drop('target')

In [33]:
# valid の分割
valid=[train.index[int(train_size*i/5):int(train_size/5*(i+1))] for i in range(k)]
pre_list=[]
for i in range(5):#CV
    #validに使う以外をtrain
    ll=np.sum([list(valid[j]) for j in range(5) if i!=j],axis=0)
    clf.fit(train.loc[ll,feature_list],train.loc[ll,'target'])
    pred=clf.predict(train.loc[valid[i],feature_list])#validに対して予測
    acc=accuracy_score(train.loc[valid[i],'target'],pred)
    pre_list.append(acc)
    print(acc)
print('ave:{}'.format(sum(pre_list)/len(pre_list)))

0.9642857142857143
0.8928571428571429
0.896551724137931
0.7857142857142857
0.896551724137931
ave:0.887192118226601


In [34]:
# train すべてで訓練し，testを評価
clf.fit(train.loc[:,feature_list],train.loc[:,'target'])
pred=clf.predict(test.loc[:,feature_list])
print(accuracy_score(test.loc[:,'target'],pred))

0.9444444444444444
